In [3]:
from functools import partial
from geopy.geocoders import Nominatim
import WazeRouteCalculator
geolocator = Nominatim(user_agent="useer_agent", domain='nominatim.openstreetmap.org')
geocode = partial(geolocator.geocode, language="it", country_codes = "ita")


In [4]:
from pandas_ods_reader import read_ods
import pandas as pd
path = "IndirizziBiblio_21-10-2019.ods"
df = read_ods(path, 'Questionari_normalizza')
df=df[df['Calendarizzata (sì/no)']=='sì']
df=df.astype({'Cap':'int32'})

In [5]:
def getLocation(name):
    geolocator = Nominatim(user_agent="useer_agent")
    geocode = partial(geolocator.geocode, language="it")
    location =geolocator.geocode(name)
    return location.latitude, location.longitude
#print(lat, lon)

In [6]:
def getName(row):
    name=row.Comune + ','
    try:
        name=name + row.Indirizzo[row.Indirizzo.lower().index('via')+3:len(row.Indirizzo)]
    except:
        try:
            name=name + row.Indirizzo[row.Indirizzo.lower().index('piazza')+6:len(row.Indirizzo)]
        except:
            try:
                name=name + row.Indirizzo[row.Indirizzo.lower().index('piazzetta')+9:len(row.Indirizzo)]
            except:
                name=name + row.Indirizzo[row.Indirizzo.lower().index('vicolo')+6:len(row.Indirizzo)]
           
    return name

In [7]:
from tqdm.notebook import tqdm as tqdm

In [19]:
import csv
x=[]
y=[""]
for i in range(len(df)):
    x.append(getName(df.iloc[i]))
x.append("Cremona, Sesto 39")

for i in range(len(df)):
    y.append(getName(df.iloc[i]))
y.append("Ostiano")
y.append("San Giovanni in Croce")
"""
with open('times.csv', mode='a') as time_file:
        time_file = csv.writer(time_file, delimiter=';')
        time_file.writerow(y)

with open('distances.csv', mode='a') as distance_file:
    distance_file = csv.writer(distance_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    distance_file.writerow(y)
"""
y.pop(0)


''

In [12]:
import time
dataf=pd.read_csv("times_computed.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
start=len(dataf)
while start<len(x):
    try:
        dataf=pd.read_csv("times_computed.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
        start=len(dataf)
        #Outer loop over cities (from)
        for j in range(start, len(x)):
            lat, lon=getLocation(x[j])
            from_address=str(lat)+", "+str(lon)
            double_error = False
            t=[x[j]]
            d=[x[j]]
            #Inner loop over cities (to)
            for i in tqdm(range(len(y)), desc=str(j)):
                wait=2
                #Loop until get distance
                while True:
                    region = 'EU'
                    lat, lon = getLocation(y[i])
                    to_address=str(lat)+", "+str(lon)
                    try:
                        route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region)
                        route_time, route_distance = route.calc_route_info(real_time=False)
                    except:
                        #On error wait (to avoid out of request) and retry to get route
                        #print(x[j]+"\n"+y[i])
                        time.sleep(wait)
                        wait=wait+2
                        continue
                    t.append(route_time)
                    d.append(route_distance)
                    break
            with open('times_computed.csv', mode='a') as time_file:
                time_file = csv.writer(time_file, delimiter=';', quoting=csv.QUOTE_MINIMAL)
                time_file.writerow(t)

            with open('distances_computed.csv', mode='a') as distance_file:
                distance_file = csv.writer(distance_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                distance_file.writerow(d)
    except:
        time.sleep(5)
        pass


In [13]:
dataf=pd.read_csv("times_computed.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
len(dataf)

67

In [15]:
lat, lon=getLocation("Bonemerse, Roma, 25")
from_address=str(lat)+", "+str(lon)
lat, lon = getLocation("Sospiro, Europa, 2")
to_address=str(lat)+", "+str(lon)
route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, "EU")
route_time, route_distance = route.calc_route_info(real_time=False)

In [56]:
import time
lat, lon=getLocation("Sospiro")
to_address=str(lat)+", "+str(lon)
t=["Sospiro"]
d=["Sospiro"]
#Inner loop over cities (to)
for i in tqdm(range(len(x))):
    wait=2
    #Loop until get distance
    while True:
        region = 'EU'
        lat, lon = getLocation(x[i])
        from_address=str(lat)+", "+str(lon)
        try:
            route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region)
            route_time, route_distance = route.calc_route_info(real_time=False)
        except:
            #On error wait (to avoid out of request) and retry to get route
            #print(x[j]+"\n"+y[i])
            time.sleep(wait)
            wait=wait+2
            continue
        t.append(route_time)
        d.append(route_distance)
        break

In [61]:
"""
with open('sospirot.csv', mode='w') as time_file:
        time_file = csv.writer(time_file, delimiter=';')
        time_file.writerow(t)
with open('sospirod.csv', mode='w') as time_file:
        time_file = csv.writer(time_file, delimiter=';')
        time_file.writerow(d)
"""

0
1
2


In [67]:
for i in t:
    with open('sospirot.csv', mode='a', newline='') as time_file:
        time_file = csv.writer(time_file)
        time_file.writerow([i])
for i in d:
    with open('sospirod.csv', mode='a', newline='') as time_file:
        time_file = csv.writer(time_file, delimiter=';')
        time_file.writerow([i])